## Task 1

In [1]:
import pandas as pd


df = pd.read_csv("tech_faq_4000_samples.csv") 


print(df)

                                  Question  \
0         What is Artificial Intelligence?   
1                What is Machine Learning?   
2                      What is Blockchain?   
3               What is Quantum Computing?   
4                   What is 5G Technology?   
...                                    ...   
3995       What is Mobile App Development?   
3996  What is Natural Language Processing?   
3997               What is Edge Computing?   
3998                 What is Data Science?   
3999              What is Computer Vision?   

                                                 Answer  
0     Artificial Intelligence refers to a technology...  
1     Machine Learning refers to a technology that f...  
2     Blockchain refers to a technology that focuses...  
3     Quantum Computing refers to a technology that ...  
4     5G Technology refers to a technology that focu...  
...                                                 ...  
3995  Mobile App Development refers to a 

## Task 2


In [2]:
from langchain.schema import Document
documents = [
    Document(page_content=row["Answer"], metadata={"question": row["Question"]})
    for _, row in df.iterrows()
]

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

In [4]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load SentenceTransformer model and HuggingFace embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the FAISS vector store
vectorstore = FAISS.from_documents(split_docs, embeddings)


c:\Users\supra\.conda\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\supra\.conda\envs\tf-gpu\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\supra\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

In [5]:
from transformers import pipeline


generator = pipeline("text2text-generation", model="google/flan-t5-large")

def generate_answer(prompt):
    result = generator(prompt, max_length=200, num_return_sequences=1)
    return result[0]["generated_text"]


c:\Users\supra\.conda\envs\tf-gpu\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\supra\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [6]:
def retrieve_documents(query, vectorstore, top_k=3):
    retrieved_docs = vectorstore.similarity_search(query, k=top_k)
    combined_content = " ".join([doc.page_content for doc in retrieved_docs])
    return combined_content, retrieved_docs


In [7]:
def answer_query(query, vectorstore):
  
    combined_content, retrieved_docs = retrieve_documents(query, vectorstore)

    # Generate answer using retrieved context
    prompt = f"Answer the question based on the following context:\n\n{combined_content}\n\nQuestion: {query}"
    answer = generate_answer(prompt)

  
    return answer, retrieved_docs


In [8]:
# Example query
query = "What is Artificial Intelligence?"
answer, source_docs = answer_query(query, vectorstore)

# Print the generated answer
print("Answer:", answer)

# Print the source documents
print("Source Documents:")
for doc in source_docs:
    print(doc.metadata["question"], ":", doc.page_content)


Answer: a technology that focuses on creating systems capable of simulating human intelligence
Source Documents:
What is Artificial Intelligence? : Artificial Intelligence refers to a technology that focuses on creating systems capable of simulating human intelligence.
What is Artificial Intelligence? : Artificial Intelligence refers to a technology that focuses on creating systems capable of simulating human intelligence.
What is Artificial Intelligence? : Artificial Intelligence refers to a technology that focuses on creating systems capable of simulating human intelligence.


## Task 3

In [9]:
def retrieve_documents(query, vectorstore, top_k=3):
    retrieved_docs = vectorstore.similarity_search(query, k=top_k)
    
    unique_docs = []
    seen_content = set()
    for doc in retrieved_docs:
        if doc.page_content not in seen_content:
            unique_docs.append(doc)
            seen_content.add(doc.page_content)
    return " ".join([doc.page_content for doc in unique_docs]), unique_docs


while True:
    query = input("Ask a question (or type 'exit' to quit): ")
    if query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    answer, source_docs = answer_query(query, vectorstore)
    print(f"\nAnswer: {answer}\n")
    print("Source Documents:")
    for doc in source_docs:
        print(f"- {doc.metadata['question']}: {doc.page_content}\n")



Answer: training algorithms to learn patterns from data

Source Documents:
- What is Machine Learning?: Machine Learning refers to a technology that focuses on training algorithms to learn patterns from data.

- What is Machine Learning?: Machine Learning refers to a technology that focuses on training algorithms to learn patterns from data.

- What is Machine Learning?: Machine Learning refers to a technology that focuses on training algorithms to learn patterns from data.

Goodbye!
